In [ ]:
import os
import openai
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file
openai.api_key = os.environ['OPENAI_API_KEY']
print(openai.api_key )
file =  os.environ["FILE_PATH"]
print(file)

In [ ]:
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import Chroma 
from langchain.text_splitter import CharacterTextSplitter
from langchain.llms import OpenAI
from langchain.chains import ConversationalRetrievalChain

In [ ]:
#Load in documents
from langchain.document_loaders import TextLoader
loader = TextLoader(file+"sample.txt")
documents = loader.load()
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
texts = text_splitter.split_documents(documents)
embeddings = OpenAIEmbeddings()
vectorstore = Chroma.from_documents(texts, embeddings)

### 创建一个内存对象，这是跟踪输入/输出和保持对话所必需的:

In [ ]:
from langchain.memory import ConversationBufferMemory
memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)

### 初始化ConversationalRetrievalChain:

In [ ]:
qa = ConversationalRetrievalChain.from_llm(OpenAI(temperature=0), vectorstore.as_retriever(), memory=memory)

In [ ]:
query = "What did the president say about Ketanji Brown Jackson"
result = qa({"question": query})

In [ ]:
result["answer"]

In [ ]:
query = "Did he mention who she suceeded"
result = qa({"question": query})

In [ ]:
result['answer']

### 不用memroy的话，也可以把历史对话放到历史对话列表里来实现：

In [ ]:
qa = ConversationalRetrievalChain.from_llm(OpenAI(temperature=0), vectorstore.as_retriever())

In [ ]:
chat_history = []
query = "What did the president say about Ketanji Brown Jackson"
result = qa({"question": query, "chat_history": chat_history})

In [ ]:
result["answer"]

In [ ]:
#把旧的query和history添加到chat_history里：
chat_history = [(query, result["answer"])]
query = "Did he mention who she suceeded"
result = qa({"question": query, "chat_history": chat_history})

In [ ]:
result['answer']